In [4]:
import torch
import math
from torch import nn
import torch.nn.functional as F

In [22]:
def attention_dot_product(q,k,v, mask = None):
    d_k = q.size()[-1]
    scaled = torch.matmul(q,k.transpose(-1,-2)/math.sqrt(d_k))
    if mask is not None:
        scaled += mask
    attention = F.softmax(scaled, dim=-1)
    values = torch.matmul(attention,v)
    return values,attention

class MultiHeadAttention(nn.Module):
    def __init__(self, vec_size, n_heads):
        super().__init__()
        self.n_heads = n_heads     #8 
        self.vec_size = vec_size    #512
        self.head_dim = vec_size//n_heads   #64
        self.qkv_layer = nn.Linear(vec_size,3*vec_size) #512x1536
        self.linear_layer = nn.Linear(vec_size,vec_size)    #512x512

    def forward(self,x,mask=None):
        batch_size, seq_len, vec_size = x.size()
        qkv = self.qkv_layer(x)
        qkv = qkv.reshape(batch_size, seq_len, self.n_heads, 3*self.head_dim)   #reshaping qkv into 8 heads(30x200x8x192)
        qkv = qkv.permute(0,2,1,3)  #30x8x200x192
        q,k,v = qkv.chunk(3, dim=-1)    #divide into 3 vectors each of 30x8x200x64
        values,attention = attention_dot_product(q,k,v,mask)    
        values = values.reshape(batch_size,seq_len,self.n_heads*self.head_dim)
        output = self.linear_layer(values)
        return output
    
class LayerNormalization(nn.Module):
    def __init__(self, parameters_shape, eps=1e-5):
        super().__init__()
        self.parameters_shape=parameters_shape
        self.eps=eps
        self.gamma = nn.Parameter(torch.ones(parameters_shape))
        self.beta =  nn.Parameter(torch.zeros(parameters_shape))

    def forward(self, inputs):
        dims = [-(i + 1) for i in range(len(self.parameters_shape))]
        mean = inputs.mean(dim=dims, keepdim=True)
        print(f"Mean ({mean.size()})")
        var = ((inputs - mean) ** 2).mean(dim=dims, keepdim=True)
        std = (var + self.eps).sqrt()
        print(f"Standard Deviation  ({std.size()})")
        y = (inputs - mean) / std
        print(f"y: {y.size()}")
        out = self.gamma * y  + self.beta
        print(f"self.gamma: {self.gamma.size()}, self.beta: {self.beta.size()}")
        print(f"out: {out.size()}")
        return out
    
class PositionwiseFeedForward(nn.Module):

    def __init__(self, vec_size, hidden, drop_prob=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(vec_size, hidden)
        self.linear2 = nn.Linear(hidden, vec_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x
    
class EncoderLayer(nn.Module):

    def __init__(self, vec_size, ffn_hidden, n_heads, drop_prob):
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(vec_size=vec_size, n_heads=n_heads)
        self.norm1 = LayerNormalization(parameters_shape=[vec_size])
        self.dropout1 = nn.Dropout(p=drop_prob)
        self.ffn = PositionwiseFeedForward(vec_size=vec_size, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm2 = LayerNormalization(parameters_shape=[vec_size])
        self.dropout2 = nn.Dropout(p=drop_prob)

    def forward(self, x):
        residual_x = x
        print("------- ATTENTION 1 ------")
        x = self.attention(x, mask=None)
        print("------- DROPOUT 1 ------")
        x = self.dropout1(x)
        print("------- ADD AND LAYER NORMALIZATION 1 ------")
        x = self.norm1(x + residual_x)
        residual_x = x
        print("------- ATTENTION 2 ------")
        x = self.ffn(x)
        print("------- DROPOUT 2 ------")
        x = self.dropout2(x)
        print("------- ADD AND LAYER NORMALIZATION 2 ------")
        x = self.norm2(x + residual_x)
        return x

class Encoder(nn.Module):
    def __init__(self, vec_size, ffn_hidden, n_heads, drop_prob, num_layers):
        super().__init__()
        self.layers = nn.Sequential(*[EncoderLayer(vec_size, ffn_hidden, n_heads, drop_prob)
                                     for _ in range(num_layers)])

    def forward(self, x):
        x = self.layers(x)
        return x

In [23]:
vec_size = 512 ##size of every vector in the architecture
n_heads = 8  ## number of attention heads
drop_prob = 0.1   ##to regularize
batch_size = 30
seq_len = 200
ffn_hidden = 2048
num_layers = 5

encoder = Encoder(vec_size, ffn_hidden, n_heads, drop_prob, num_layers)

In [24]:
x = torch.randn( (batch_size, seq_len, vec_size) ) # includes positional encoding
out = encoder(x)

------- ATTENTION 1 ------
------- DROPOUT 1 ------
------- ADD AND LAYER NORMALIZATION 1 ------
Mean (torch.Size([30, 200, 1]))
Standard Deviation  (torch.Size([30, 200, 1]))
y: torch.Size([30, 200, 512])
self.gamma: torch.Size([512]), self.beta: torch.Size([512])
out: torch.Size([30, 200, 512])
------- ATTENTION 2 ------
------- DROPOUT 2 ------
------- ADD AND LAYER NORMALIZATION 2 ------
Mean (torch.Size([30, 200, 1]))
Standard Deviation  (torch.Size([30, 200, 1]))
y: torch.Size([30, 200, 512])
self.gamma: torch.Size([512]), self.beta: torch.Size([512])
out: torch.Size([30, 200, 512])
------- ATTENTION 1 ------
------- DROPOUT 1 ------
------- ADD AND LAYER NORMALIZATION 1 ------
Mean (torch.Size([30, 200, 1]))
Standard Deviation  (torch.Size([30, 200, 1]))
y: torch.Size([30, 200, 512])
self.gamma: torch.Size([512]), self.beta: torch.Size([512])
out: torch.Size([30, 200, 512])
------- ATTENTION 2 ------
------- DROPOUT 2 ------
------- ADD AND LAYER NORMALIZATION 2 ------
Mean (tor